<a href="https://colab.research.google.com/github/helene-78/Accidents/blob/main/prep_df_utiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE # Pour avoir 50 % de 0 et 50 % de 1 dans la colonne presence_accident
from sklearn.linear_model import LogisticRegression # Regression logistique
from sklearn import metrics
from sklearn.model_selection import RepeatedKFold #KFold répété, pour 
from sklearn.model_selection import train_test_split

**Fusion des différents dataframes**

In [2]:
df_accidents = pd.read_csv('caracteristiques-2019.csv', sep=";")
df_usagers = pd.read_csv('usagers-2019.csv', sep=";")
df_lieux = pd.read_csv('lieux-2019.csv', sep=";")
df_vehicules = pd.read_csv('vehicules-2019.csv', sep=";")
df_test = df_accidents.merge(df_usagers.merge(df_lieux.merge(df_vehicules))) #dataframe à traiter

**Pour supprimer les colonnes inutiles à l'analyse de données, supprimer les NaN et avoir une colonne heure au bon format**

In [3]:
df_utiles = df_test.drop(axis=1, labels=['Num_Acc', 'com', 'adr', 'v1', 'id_vehicule', 'num_veh', 'lartpc', 'larrout', 'occutc', 'v2', 'voie'])
df_utiles = df_utiles.dropna()
df_utiles["hrmn"] = pd.to_datetime(df_utiles["hrmn"], format='%H:%S')
df_utiles["hrmn"] = df_utiles["hrmn"].apply(lambda x: int(x.hour)*60 + int(x.minute))
df_utiles.dtypes

jour        int64
mois        int64
an          int64
hrmn        int64
lum         int64
dep        object
agg         int64
int         int64
atm         int64
col         int64
lat        object
long       object
place       int64
catu        int64
grav        int64
sexe        int64
an_nais     int64
trajet      int64
secu1       int64
secu2       int64
secu3       int64
locp        int64
actp       object
etatp       int64
catr        int64
circ        int64
nbv         int64
vosp        int64
prof        int64
pr         object
pr1        object
plan        int64
surf        int64
infra       int64
situ        int64
vma         int64
senc        int64
catv        int64
obs         int64
obsm        int64
choc        int64
manv        int64
motor       int64
dtype: object

In [4]:
df_utiles.head()

,jour,mois,an,hrmn,lum,dep,agg,int,atm,col,...,infra,situ,vma,senc,catv,obs,obsm,choc,manv,motor
0,30,11,2019,60,4,93,1,1,1,2,...,2,1,70,2,7,0,2,5,23,1
1,30,11,2019,60,4,93,1,1,1,2,...,2,1,70,2,7,0,2,5,23,1
2,30,11,2019,60,4,93,1,1,1,2,...,2,1,70,2,17,1,0,3,11,1
3,30,11,2019,120,3,93,1,1,1,6,...,0,1,70,1,7,4,0,1,0,1
4,28,11,2019,900,1,92,1,1,1,4,...,0,1,90,1,7,0,2,1,2,1


**Création et remplissage d'une colonne "presence_deces" qui vaut 1 en cas d'accident mortel, 0 sinon**

In [5]:
df_utiles["hrmn"] = df_utiles["hrmn"].astype('int64')
df_utiles["dep"] = df_utiles["dep"].replace(["2B"], "999") #On remplace les codes des départements de la Corse par des int
df_utiles["dep"] = df_utiles["dep"].replace(["2A"], "998")
df_utiles["actp"] = df_utiles["actp"].replace(["A"], "14") #On remplace les actions du piéton non encodées par des chiffres
df_utiles["actp"] = df_utiles["actp"].replace(["B"], "15")
df_utiles["dep"] = df_utiles["dep"].astype('int64')
df_utiles["grav"] = df_utiles["grav"].apply(lambda x: int(x==2)) # On recode la colonne "grav" : tué = 1, blessé léger/hospitalisé/indemne = 0
df_utiles.to_csv('df_utiles.csv')